In [2]:
!pip install transformers
!pip install transformers[sentencepiece]

     |████████████████████████████████| 3.1 MB 4.1 MB/s 
     |████████████████████████████████| 59 kB 5.6 MB/s 
     |████████████████████████████████| 895 kB 48.0 MB/s 
     |████████████████████████████████| 596 kB 56.7 MB/s 
     |████████████████████████████████| 3.3 MB 26.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.4 MB/s 


In [36]:

!pip install transformers
!pip install transformers[sentencepiece]

import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

# torch.manual_seed(42) to produce same results

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)





def generate_similar_questions(question):
    model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
    tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # print ("device ",device)
    model = model.to(device)
    
    # prompt design
    text =  "paraphrase: " + question + " </s>"


    max_len = 256

    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)



    beam_outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
        max_length=256,
        top_k=80,
        top_p=0.96,
        early_stopping=True,
        num_return_sequences=3,
        temperature = 0.65
    )

    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)

    return final_outputs

In [37]:
question_list = ['What are the factors driving the advanced analytics market growth?',
'How big is the advanced analytics market?',
'What is advanced analytics market growth?',
'Which type segment accounted for the highest advanced analytics market share?',
'Who are the key players in the advanced analytics market?',
'Which deployment segment held the largest advanced analytics market share?', 
'Which enterprise size segment led the advanced analytics market?', 
'Which end-use segment dominated the advanced analytics market?', 
'Which region held the highest advanced analytics market share?']


for question in question_list:
    print(question)
    print('Similar_Questions: ',generate_similar_questions(question),'\n\n' )

What are the factors driving the advanced analytics market growth?


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Similar_Questions:  ['What factors are driving the advanced analytics market growth?'] 


How big is the advanced analytics market?
Similar_Questions:  ['What is the advanced analytics market?', 'How big is the advanced analytics market?', 'What is the market for advanced analytics?'] 


What is advanced analytics market growth?
Similar_Questions:  ['What is advanced analytics market growth?'] 


Which type segment accounted for the highest advanced analytics market share?
Similar_Questions:  ['Which type of advanced analytics market share is the highest?', 'Which type of advanced analytics is the most lucrative segment in the world?'] 


Who are the key players in the advanced analytics market?
Similar_Questions:  ['Who are the key players in the Advanced Analytics market?', 'Who is the biggest player in the advanced analytics market?', 'Who are the key players in advanced analytics market?'] 


Which deployment segment held the largest advanced analytics market share?
Similar_Questio

In [57]:
!pip freeze > requirements.txt

In [28]:
import torch
import numpy as np
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import AutoModelForSequenceClassification, AutoTokenizer


class BertSquad(object):

    def __init__(self):

        self.USE_SUMMARY = False

        self.QA_MODEL = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad') # where are the models
        self.QA_TOKENIZER = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
        self.torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.QA_MODEL.to(self.torch_device)
        self.QA_MODEL.eval()


        self.MNLI_MODEL = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
        self.MNLI_TOKENIZER = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')


        if self.USE_SUMMARY:
            self.SUMMARY_TOKENIZER = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
            self.SUMMARY_MODEL = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
            self.SUMMARY_MODEL.to(self.torch_device)
            self.SUMMARY_MODEL.eval()

        
    def reconstructText(self,tokens, start=0, stop=-1):
        tokens = tokens[start: stop]
        if '[SEP]' in tokens:
            sepind = tokens.index('[SEP]')
            tokens = tokens[sepind+1:]
        txt = ' '.join(tokens)
        txt = txt.replace(' ##', '')
        txt = txt.replace('##', '')
        txt = txt.strip()
        txt = " ".join(txt.split())
        txt = txt.replace(' .', '.')
        txt = txt.replace(' . ', '.')
        txt = txt.replace('( ', '(')
        txt = txt.replace(' )', ')')
        txt = txt.replace(' - ', '-')
        txt_list = txt.split(' , ')
        txt = ''
        nTxtL = len(txt_list)
        if nTxtL == 1:
            return txt_list[0]
        newList =[]
        for i,t in enumerate(txt_list):
            if i < nTxtL -1:
                if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                    newList += [t,',']
                else:
                    newList += [t, ', ']
            else:
                newList += [t]
        return ''.join(newList)


    def makeBERTSQuADPrediction(self,document, question):
        ## we need to rewrite this function so that it chuncks the document into 250-300 word segments with
        ## 50 word overlaps on either end so that it can understand and check longer abstracts
        nWords = len(document.split())
        input_ids_all = self.QA_TOKENIZER.encode(question, document)
        tokens_all = self.QA_TOKENIZER.convert_ids_to_tokens(input_ids_all)
        overlapFac = 1.1
        # print(input_ids_all)

        if len(input_ids_all)*overlapFac > 2048:
            nSearchWords = int(np.ceil(nWords/5))
            quarter = int(np.ceil(nWords/4))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[quarter-int(nSearchWords*overlapFac/2):quarter+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[quarter*2-int(nSearchWords*overlapFac/2):quarter*2+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[quarter*3-int(nSearchWords*overlapFac/2):quarter*3+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [self.QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
            
        elif len(input_ids_all)*overlapFac > 1536:
            nSearchWords = int(np.ceil(nWords/4))
            third = int(np.ceil(nWords/3))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[third-int(nSearchWords*overlapFac/2):third+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[third*2-int(nSearchWords*overlapFac/2):third*2+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [self.QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
            
        elif len(input_ids_all)*overlapFac > 1024:
            nSearchWords = int(np.ceil(nWords/3))
            middle = int(np.ceil(nWords/2))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[middle-int(nSearchWords*overlapFac/2):middle+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [self.QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        
        elif len(input_ids_all)*overlapFac > 512:
            nSearchWords = int(np.ceil(nWords/2))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [self.QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        else:
            input_ids = [input_ids_all]
        absTooLong = False    
        
        answers = []
        cons = []

        for iptIds in input_ids:
            tokens = self.QA_TOKENIZER.convert_ids_to_tokens(iptIds)
            sep_index = iptIds.index(self.QA_TOKENIZER.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(iptIds) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b
            assert len(segment_ids) == len(iptIds)
            n_ids = len(segment_ids)

            if n_ids < 512:
                res = self.QA_MODEL(torch.tensor([iptIds]).to(self.torch_device), 
                                        token_type_ids=torch.tensor([segment_ids]).to(self.torch_device))
                
                start_scores = res.start_logits
                end_scores = res.end_logits

              
            else:
                
                print('****** warning only considering first 512 tokens, document is '+str(nWords)+' words long.  There are '+str(n_ids)+ ' tokens')
                absTooLong = True
                res = self.QA_MODEL(torch.tensor([iptIds[:512]]).to(self.torch_device), 
                                        token_type_ids=torch.tensor([segment_ids[:512]]).to(self.torch_device),)
                start_scores = res.start_logits
                end_scores = res.end_logits
                
            
            start_scores = start_scores[:,1:-1]
            end_scores = end_scores[:,1:-1]
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores)
           
            answer = self.reconstructText(tokens, answer_start, answer_end+4)
        
            if answer.startswith('.') or answer.startswith(','):
                answer = answer[2:]
                
            c = start_scores[0,answer_start].item()+end_scores[0,answer_end].item()
            answers.append(answer)
            cons.append(c)
        
        maxC = max(cons)
        iMaxC = [i for i, j in enumerate(cons) if j == maxC][0]
        confidence = cons[iMaxC]
        answer = answers[iMaxC]
        
        sep_index = tokens_all.index('[SEP]')
        full_txt_tokens = tokens_all[sep_index+1:]
        
        abs_returned = self.reconstructText(full_txt_tokens)

        ans={}
        ans['answer'] = answer
        
        if answer.startswith('[CLS]') or answer_end.item() < sep_index or answer.endswith('[SEP]'):
            ans['confidence'] = -1000000
        else:
            ans['confidence'] = confidence
        
        ans['abstract_bert'] = abs_returned
        ans['abs_too_long'] = absTooLong
        
        
        # mnli_response = self.get_mnli_response(context,question)
        # ans.update(mnli_response)
        return ans

    def get_mnli_response(self,document,question):
        result = {}

        premise = document
        hypothesis = question

        # run through model pre-trained on MNLI
        x = self.MNLI_TOKENIZER.encode(premise, hypothesis, return_tensors='pt')
        logits = self.MNLI_MODEL(x.to(self.torch_device))[0]

        # output = tokenizer.decode(logits)
        labels = ['contradiction','entailment']

        # we throw away "neutral" (dim 1) and take the probability of
        # "entailment" (2) as the probability of the label being true 
        entail_contradiction_logits = logits[:,[0,2]]
        probs = entail_contradiction_logits.softmax(dim=1)
        prob_label_is_true = probs[:,1]
        label = labels[np.argmax(probs[0].tolist())]
        result['label'] = label
        result['mnli_score'] = max(probs[0].tolist())
        return result

bq = BertSquad()

# ans = bq.makeBERTSQuADPrediction()

In [ ]:
 if self.USE_SUMMARY:
            allAnswersTxt = ' '.join(ranked_aswers[:6]).replace('\n','')
            answers_input_ids = self.SUMMARY_TOKENIZER.batch_encode_plus([allAnswersTxt], return_tensors='pt', max_length=1024)['input_ids'].to(self.torch_device)
            summary_ids = self.SUMMARY_MODEL.generate(answers_input_ids, num_beams=10, length_penalty=1.2, max_length=1024, min_length=64, no_repeat_ngram_size=4)

            exec_sum = self.SUMMARY_TOKENIZER.decode(summary_ids.squeeze(), skip_special_tokens=True)
            execSum_HTML = '<div style="font-size:12px;color:#CCCC00"><b>BART Abstractive Summary:</b>: '+exec_sum+'</div>'
            warning_HTML = '<div style="font-size:12px;padding-bottom:12px;color:#CCCC00;margin-top:1px"> Warning: This is an autogenerated summary based on semantic search of abstracts, please examine the results before accepting this conclusion. There may be scenarios in which the summary will not be able to clearly answer the question.</div>'
 

In [58]:
hit_dictionary = [
  {
    "Account Name": "Plantpick",
    "Opportunity Name": "Plantpick - Smartphone consumer review analysis - Phase#1",
    "Opportunity Owner": "Jitendra Jethanandani",
    "P&L BU": "Enterprise Tech",
    "Account Type": "D",
    "Stage": "Closed Won",
    "Bi2i Industry": "HiTech Enterprise",
    "Bi2i Sub-Industry": "B2B Tech",
    "Created Date": "4/24/2019",
    "Last Stage Change Date": "5/15/2019",
    "Probability (%)": 100,
    "Sum of Amount": 61800,
    "Sum of Expected Revenue": 61800,
    "values": "Account Name Plantpick Opportunity Name Plantpick - Smartphone consumer review analysis - Phase#1 P&L BU Enterprise Tech Account Type D Bi2i Industry HiTech Enterprise Bi2i Sub-Industry B2B Tech Sum of Amount 61800.0 Sum of Expected Revenue 61800.0",
    "source": "Dataset",
    "confidence": 7.7573066
  },
  {
    "source": "PDF documents",
    "creationdate": "11/16/21, 12:54 PM ",
    "topichead": "Global Advanced Analytics Market Size Report, 2021-2028",
    "contentpage": "2/9",
    "urls": [
      "https://www.grandviewresearch.com/industry-analysis/artificial-intelligence-ai-market),",
      "https://www.grandviewresearch.com/industry-analysis/advanced-analytics-market"
    ],
    "values": "To learn more about this report request a free sample copy industry-analysis advanced-analytics-market request rs2 The volume of unorganized data such as surveillance data corporate emails and consumer data generated from streaming activity for media subscription services such as Amazon Prime Video NetFlix and Hulu has been rising in recent years. Hence the demand for database management tools has also been rising. This has particularly translated into the rising demand for analytics for data management and strategic decision-making. At the same time the continued integration of analytics with Geographical Information System GIS and improvements in location-based services have increased the volumes of geospatial data significantly thereby necessitating the adoption of advanced analytics for optimal management of the data. Several businesses are adopting progressive analytics solutions which can potentially utilize geospatial data to locate clients on a map and draft new strategies accordingly. The growing adoption of advanced analytics in predicting and forecasting trading patterns electricity consumption patterns and traffic conditions is propelling the growth of the market. The use of analytics in demand forecasting allows organizations to make data-based informed decisions and augment profitability. Apart from incumbents of various industries and industry verticals such as manufacturing banking and professional services government agencies are also investing aggressively in big data analytics thereby driving the demand for advanced analytics. The rising demand for IoT-based predictive solutions which can potentially help in collecting and analyzing data from sensors including temperature sensors air quality sensors and motion sensors among others within the IoT network is expected to boost the adoption of advanced analytical tools over the forecast period. Type Insights The big data analytics segment accounted for the largest share of around 30% in 2020 owing to the increasing popularity of social media and the rise in the number of virtual or digital offices that produce large volumes of data. Information management is emerging as an area where big data analytics can have a significant impact on business processes and productivity. The growing adoption of big data analytics across businesses for enhancing operational efficiency and strengthening market strategies is expected to drive the growth of the segment. The customer analytics segment is expected to grow significantly over the forecast period in line with the increasing demand for improved lead management customer retention and customer experience management. The strong emphasis businesses are putting on understanding the customers buying patterns and behavior and accordingly offering a highly customizable customer experience is driving the demand for customer analytics. Continued introduction of business process automation advances in the latest technologies such as machine learning and artificial intelligence  www.grandviewresearch.com industry-analysis artificial-intelligence-ai-market and the integration of these technologies into customer analytics are expected to contribute to the growth of the segment. Efforts being pursued by the incumbents of the retail industry to offer an omnichannel experience to their customers and the growing adoption of customer analytics by large retailers to roll out marketing programs and personalized communications also bode well for the growth of the segment. This site uses cookies to improve user",
    "confidence": 2.464856
  },
  {
    "Account Name": "Cellular Curvaceous",
    "Opportunity Name": "Cellular CurvaceousFPA",
    "Opportunity Owner": "Mukesh Saharan",
    "P&L BU": "Consumer",
    "Account Type": "A",
    "Stage": "Solutioning",
    "Bi2i Industry": "Consumer",
    "Bi2i Sub-Industry": "CPG",
    "Created Date": 43414,
    "Last Stage Change Date": "(blank)",
    "Probability (%)": 70,
    "Sum of Amount": 80000,
    "Sum of Expected Revenue": 56000,
    "values": "Account Name Cellular Curvaceous Opportunity Name Cellular CurvaceousFPA P&L BU Consumer Account Type A Bi2i Industry Consumer Bi2i Sub-Industry CPG Sum of Amount 80000.0 Sum of Expected Revenue 56000.0",
    "source": "Dataset",
    "confidence": 2.2711976
  },
  {
    "Account Name": "Sigmasocial",
    "Opportunity Name": "Sigmasocial - Commodity Forecasting",
    "Opportunity Owner": "Arun Krishnamoorthy",
    "P&L BU": "Consumer",
    "Account Type": "A",
    "Stage": "Closed Won",
    "Bi2i Industry": "Consumer",
    "Bi2i Sub-Industry": "CPG",
    "Created Date": "3/19/2019",
    "Last Stage Change Date": "9/29/2019",
    "Probability (%)": 100,
    "Sum of Amount": 48387,
    "Sum of Expected Revenue": 48387,
    "values": "Account Name Sigmasocial Opportunity Name Sigmasocial - Commodity Forecasting P&L BU Consumer Account Type A Bi2i Industry Consumer Bi2i Sub-Industry CPG Sum of Amount 48387.0 Sum of Expected Revenue 48387.0",
    "source": "Dataset",
    "confidence": 2.2711976
  },
  {
    "Account Name": "Sigmasocial",
    "Opportunity Name": "Sigmasocial - Jarvis USA",
    "Opportunity Owner": "Sonal Gupta",
    "P&L BU": "Consumer",
    "Account Type": "A",
    "Stage": "Closed Won",
    "Bi2i Industry": "Consumer",
    "Bi2i Sub-Industry": "CPG",
    "Created Date": "11/21/2019",
    "Last Stage Change Date": 43781,
    "Probability (%)": 100,
    "Sum of Amount": 34650,
    "Sum of Expected Revenue": 34650,
    "values": "Account Name Sigmasocial Opportunity Name Sigmasocial - Jarvis USA P&L BU Consumer Account Type A Bi2i Industry Consumer Bi2i Sub-Industry CPG Sum of Amount 34650.0 Sum of Expected Revenue 34650.0",
    "source": "Dataset",
    "confidence": 2.2711976
  },
  {
    "Account Name": "Sigmasocial",
    "Opportunity Name": "Sigmasocial Jobby MDM",
    "Opportunity Owner": "Sonal Gupta",
    "P&L BU": "Consumer",
    "Account Type": "A",
    "Stage": "Negotiation/Review",
    "Bi2i Industry": "Consumer",
    "Bi2i Sub-Industry": "CPG",
    "Created Date": 43716,
    "Last Stage Change Date": "10/25/2021",
    "Probability (%)": 90,
    "Sum of Amount": 15000,
    "Sum of Expected Revenue": 13500,
    "values": "Account Name Sigmasocial Opportunity Name Sigmasocial Jobby MDM P&L BU Consumer Account Type A Bi2i Industry Consumer Bi2i Sub-Industry CPG Sum of Amount 15000.0 Sum of Expected Revenue 13500.0",
    "source": "Dataset",
    "confidence": 2.2711976
  },
  {
    "Account Name": "Sigmasocial",
    "Opportunity Name": "Sigmasocial - Jarvis India Support",
    "Opportunity Owner": "Sonal Gupta",
    "P&L BU": "Consumer",
    "Account Type": "A",
    "Stage": "Closed Won",
    "Bi2i Industry": "Consumer",
    "Bi2i Sub-Industry": "CPG",
    "Created Date": "9/30/2019",
    "Last Stage Change Date": "11/18/2019",
    "Probability (%)": 100,
    "Sum of Amount": 72300,
    "Sum of Expected Revenue": 72300,
    "values": "Account Name Sigmasocial Opportunity Name Sigmasocial - Jarvis India Support P&L BU Consumer Account Type A Bi2i Industry Consumer Bi2i Sub-Industry CPG Sum of Amount 72300.0 Sum of Expected Revenue 72300.0",
    "source": "Dataset",
    "confidence": 2.2609577
  },
  {
    "Account Name": "Sigmasocial",
    "Opportunity Name": "Sigmasocial - Jarvis Phase 2",
    "Opportunity Owner": "Sonal Gupta",
    "P&L BU": "Consumer",
    "Account Type": "A",
    "Stage": "Closed Won",
    "Bi2i Industry": "Consumer",
    "Bi2i Sub-Industry": "CPG",
    "Created Date": "3/14/2019",
    "Last Stage Change Date": 43472,
    "Probability (%)": 100,
    "Sum of Amount": 75000,
    "Sum of Expected Revenue": 75000,
    "values": "Account Name Sigmasocial Opportunity Name Sigmasocial - Jarvis Phase 2 P&L BU Consumer Account Type A Bi2i Industry Consumer Bi2i Sub-Industry CPG Sum of Amount 75000.0 Sum of Expected Revenue 75000.0",
    "source": "Dataset",
    "confidence": 2.2609577
  },
  {
    "Account Name": "Sigmasocial",
    "Opportunity Name": "Sigmasocial Finance Control Watchtower",
    "Opportunity Owner": "Sonal Gupta",
    "P&L BU": "Consumer",
    "Account Type": "A",
    "Stage": "Closed Won",
    "Bi2i Industry": "Consumer",
    "Bi2i Sub-Industry": "CPG",
    "Created Date": "3/19/2019",
    "Last Stage Change Date": "9/29/2019",
    "Probability (%)": 100,
    "Sum of Amount": 15714,
    "Sum of Expected Revenue": 15714,
    "values": "Account Name Sigmasocial Opportunity Name Sigmasocial Finance Control Watchtower P&L BU Consumer Account Type A Bi2i Industry Consumer Bi2i Sub-Industry CPG Sum of Amount 15714.0 Sum of Expected Revenue 15714.0",
    "source": "Dataset",
    "confidence": 2.2609577
  },
  {
    "Account Name": "Cellular Curvaceous",
    "Opportunity Name": "Cellular CurvaceousAgent performance Assessment",
    "Opportunity Owner": "Mukesh Saharan",
    "P&L BU": "Consumer",
    "Account Type": "A",
    "Stage": "Closed Won",
    "Bi2i Industry": "Consumer",
    "Bi2i Sub-Industry": "CPG",
    "Created Date": "11/13/2018",
    "Last Stage Change Date": 43525,
    "Probability (%)": 100,
    "Sum of Amount": 117222,
    "Sum of Expected Revenue": 117222,
    "values": "Account Name Cellular Curvaceous Opportunity Name Cellular CurvaceousAgent performance Assessment P&L BU Consumer Account Type A Bi2i Industry Consumer Bi2i Sub-Industry CPG Sum of Amount 117222.0 Sum of Expected Revenue 117222.0",
    "source": "Dataset",
    "confidence": 2.2508814
  }
]

In [55]:
from IPython.core.display import display, HTML
from scipy.stats import norm


def highlight_answer(answer_dict):
    start_idx = ans['abstract_bert'].index(ans['answer'])
    end_idx = start_idx+len(ans['answer'])
    reframe_text = ans['abstract_bert'][:start_idx-1] + "  <font color='#1E90FF'> <b>" + ans['abstract_bert'][start_idx:end_idx] + " </b></font> " + ans['abstract_bert'][end_idx+1 : ]
    return reframe_text

def show_query(query):
        """HTML print format for the searched query"""
        return HTML('<br/><div font-size: 20px;'
                    'padding-bottom:12px"><b>Query</b>: ' + query + '</div>')



def get_ranked_answers(question,hit_dictionary):
    answers_list = []
    
    for doc in hit_dictionary:
        text = doc['values']
        ans = bq.makeBERTSQuADPrediction(text,question)
        
        if ans['answer'] != '':
            ans.update({'hl_answer' : highlight_answer(ans)})
            # Creating an weighted Average
            norm_score = (0.65*ans['confidence']+ 0.45*doc['confidence'] )/2
            ans.update({'norm_score':norm_score})
            answers_list.append(ans)
    
    answers_list = sorted(answers_list, key = lambda i: i['norm_score'],reverse=True)
    print(answers_list)


    
   



In [56]:
answers_list = get_answers_list(question,hit_dictionary)
answers_list

[{'answer': 'the big data analytics segment accounted', 'confidence': 15.292038440704346, 'abstract_bert': 'to learn more about this report request a free sample copy industry-analysis advanced-analytics-market request rs2 the volume of unorganized data such as surveillance data corporate emails and consumer data generated from streaming activity for media subscription services such as amazon prime video netflix and hulu has been rising in recent years. hence the demand for database management tools has also been rising. this has particularly translated into the rising demand for analytics for data management and strategic decision-making. at the same time the continued integration of analytics with geographical information system gis and improvements in location-based services have increased the volumes of geospatial data significantly thereby necessitating the adoption of advanced analytics for optimal management of the data. several businesses are adopting progressive analytics solu

In [60]:

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

answer_start_idx = ans['abstract_bert'].index(ans['answer'])
# print(answer_start_idx)

for doc in hit_dictionary:
        all_context = []
        # if doc['source'] == 'PDF documents':
        text = doc['values']
        #print(text)
        sentance_start_idx_list = find(text,'.')
        sentance_start_idx_list = sentance_start_idx_list + [answer_start_idx]
        sentance_start_idx_list = sorted(sentance_start_idx_list)
        sentance_start_idx = sentance_start_idx_list[find(sentance_start_idx_list,answer_start_idx)[0] - 1]
        sentance_end_idx = sentance_start_idx_list [find(sentance_start_idx_list,answer_start_idx)[0] + 1]
        context = text[sentance_start_idx:sentance_end_idx]
        all_context.append(context)

summarization_input = ' .'.join(all_context)   
print(summarization_input)       

    # print(text)

In [8]:
question = 'Which type segment accounted for the highest advanced analytics market share?' #12 secs #without MNLI 7secs
context = 'India Japan Latin America Brazil Middle East Africa MEA Frequently Asked Questions About This Report What are the factors driving the advanced analytics market growth How big is the advanced analytics market What is advanced analytics market growth Which type segment accounted for the highest advanced analytics market share Who are the key players in the advanced analytics market Which deployment segment held the largest advanced analytics market share Which enterprise size segment led the advanced analytics market Which end-use segment dominated the advanced analytics market Which region held the highest advanced analytics market share Key questions answered by the report Request a Free Sample industry-analysis advanced-analytics-market request rs7 Share www.linkedin.com shareArticle mini true url  www.grandviewresearch.com industry-analysis advanced-analytics- market title Advanced Analytics Market Size%2C Share %26 Trends Analysis Report By Type %28Big Data Analytics%2C Business Analytics%29%2C By Deployment %28On- premise%2C Cloud%29%2C By Enterprise Size%2C By End-use%2C By Region%2C And Segment Forecasts%2C 2021 - 2028 submitted-image- url www.grandviewresearch.com static img logo.svg source  www.grandviewresearch.com E-mail Save Print GET A FREE SAMPLE This FREE sample includes market data points ranging from trend analyses to market estimates forecasts. See for yourself... SEND ME A FREE SAMPLE This site uses cookies to improve user industry-analysis advanced-analytics-market request rs15 experience. More Info info privacy-policy' 
ans = bq.makeBERTSQuADPrediction(context,question)

In [67]:
from IPython.core.display import display, HTML

def show_query(query):
        """HTML print format for the searched query"""
        return HTML('<br/><div font-size: 20px;'
                    'padding-bottom:12px"><b>Query</b>: ' + query + '</div>')

In [5]:
ans

{'abs_too_long': False,
 'abstract_bert': 'india japan latin america brazil middle east africa mea frequently asked questions about this report what are the factors driving the advanced analytics market growth how big is the advanced analytics market what is advanced analytics market growth which type segment accounted for the highest advanced analytics market share who are the key players in the advanced analytics market which deployment segment held the largest advanced analytics market share which enterprise size segment led the advanced analytics market which end-use segment dominated the advanced analytics market which region held the highest advanced analytics market share key questions answered by the report request a free sample industry-analysis advanced-analytics-market request rs7 share www. linkedin. com sharearticle mini true url www. grandviewresearch. com industry-analysis advanced-analytics-market title advanced analytics market size % 2c share % 26 trends analysis repo

In [42]:
ans['abstract_bert'].index(ans['answer'])

383

In [54]:

def highlight_answer(answer_dict):
    start_idx = ans['abstract_bert'].index(ans['answer'])
    end_idx = start_idx+len(ans['answer'])
    reframe_text = ans['abstract_bert'][:start_idx-1] + "  <font color='#1E90FF'> <b>" + ans['abstract_bert'][start_idx:end_idx] + " </b></font> " + ans['abstract_bert'][end_idx+1 : ]
    return HTML(reframe_text)

In [40]:
show_query(question)

In [69]:
highlight_answer(ans)

In [19]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

torch_device ='cpu'

import numpy as np
def get_mnli_response(document,question):
    result = {}

    premise = document
    hypothesis = question

    # run through model pre-trained on MNLI
    x = tokenizer.encode(premise, hypothesis, return_tensors='pt')
    logits = nli_model(x.to(torch_device))[0]

    # output = tokenizer.decode(logits)
    labels = ['contradiction','entailment']

    # we throw away "neutral" (dim 1) and take the probability of
    # "entailment" (2) as the probability of the label being true 
    entail_contradiction_logits = logits[:,[0,2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs[:,1]
    label = labels[np.argmax(probs[0].tolist())]
    result['label'] = label
    result['mnli_score'] = max(probs[0].tolist())
    return result


mnli_result = get_mnli_response(context,question)
mnli_result

{'label': 'entailment', 'mnli_score': 0.6221316456794739}

In [24]:
res = ans.update(mnli_result)
res

In [26]:
ans['']

{'abs_too_long': False,
 'abstract_bert': 'account name plantpick opportunity name plantpick-smartphone consumer review analysis-phase # 1 p & l bu enterprise tech account type d bi2i industry hitech enterprise bi2i sub-industry b2b tech sum of amount 61800. 0 sum of expected revenue 61800. 0',
 'answer': '',
 'confidence': 8.318978786468506,
 'label': 'entailment',
 'mnli_score': 0.6221316456794739}

In [31]:
def get_final_answer(result_dict):

    norm_score = np.log(result_dict['confidence'] + result_dict['mnli_score'])
    result_dict['norm_score'] = norm_score
    return result_dict

In [30]:
get_final_answer(ans)

{'abs_too_long': False,
 'abstract_bert': 'account name plantpick opportunity name plantpick-smartphone consumer review analysis-phase # 1 p & l bu enterprise tech account type d bi2i industry hitech enterprise bi2i sub-industry b2b tech sum of amount 61800. 0 sum of expected revenue 61800. 0',
 'answer': '',
 'confidence': 8.318978786468506,
 'label': 'entailment',
 'mnli_score': 0.6221316456794739,
 'norm_score': 1.6439359456536249}

In [1]:
!pip install transformers


     |████████████████████████████████| 3.1 MB 6.4 MB/s 
     |████████████████████████████████| 596 kB 68.0 MB/s 
     |████████████████████████████████| 3.3 MB 52.0 MB/s 
     |████████████████████████████████| 895 kB 57.9 MB/s 
     |████████████████████████████████| 59 kB 8.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [44]:
from transformers import pipeline

class QuestionDatasetIndentifier(object):
    def __init__(self):
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    def get_dataset_label(self,question):
        candidate_labels = ['Gartner Market Reports', 'Inter-Company Reports']
        dataset_label = {'Gartner Market Reports':'Gartner','Inter-Company Reports':'sfdcdata' }

        classifier_response =  self.classifier(question, candidate_labels)
        predicted_label = classifier_response['labels'][0]
        label = dataset_label.get(predicted_label)
        return label



In [45]:
q_identifer = QuestionDatasetIndentifier()


In [46]:
for q in question_list:
    q_label = q_identifer.get_dataset_label(q)
    print(q_label)

Gartner
Gartner
Gartner
sfdcdata
Gartner
sfdcdata
Gartner
sfdcdata
Gartner
sfdcdata
sfdcdata
sfdcdata
sfdcdata
sfdcdata
sfdcdata
Gartner


In [5]:
sequence_to_classify = "How is the advanced analytics market is expected to grow ?"

# sequence_to_classify = 'What is my revenue growth?'

candidate_labels = ['Market Reports', 'SFDC database']
classifier(sequence_to_classify, candidate_labels)

{'labels': ['Market Reports', 'SFDC database'],
 'scores': [0.5086444616317749, 0.4913555681705475],
 'sequence': 'How is the advanced analytics market is expected to grow ?'}

In [42]:
question_list = [
"How is the advanced analytics market is expected to grow ?",                 
'What are the factors driving the advanced analytics market growth?',
'How big is the advanced analytics market?',
'What is advanced analytics market growth?',
'Which type segment accounted for the highest advanced analytics market share?',
'Who are the key players in the advanced analytics market?',
'Which deployment segment held the largest advanced analytics market share?', 
'Which enterprise size segment led the advanced analytics market?', 
'Which end-use segment dominated the advanced analytics market?', 
'Which region held the highest advanced analytics market share?',
'What is my Revenue growth rate?',
'what are our new projects won?',
'how are we expected to grow?',
'what is growth by verticals?',
'what is my split?',
'What is Market Forecast?'
]



In [38]:
candidate_labels = ['Gartner Market Reports', 'Inter-Company Reports']
dataset_label = {'Gartner Market Reports':'Gartner','Inter-Company Reports':'sfdcdata' }

for question in question_list:
   classifier_response =  classifier(question, candidate_labels)
   print(question)
   predicted_label = classifier_response['labels'][0]
   label = dataset_label.get(predicted_label)
   print(label,'\n\n')



How is the advanced analytics market is expected to grow ?
Gartner 


What are the factors driving the advanced analytics market growth?
Gartner 


How big is the advanced analytics market?
Gartner 


What is advanced analytics market growth?
sfdcdata 


Which type segment accounted for the highest advanced analytics market share?
Gartner 


Who are the key players in the advanced analytics market?
sfdcdata 


Which deployment segment held the largest advanced analytics market share?
Gartner 


Which enterprise size segment led the advanced analytics market?
sfdcdata 


Which end-use segment dominated the advanced analytics market?
Gartner 


Which region held the highest advanced analytics market share?
sfdcdata 


What is my Revenue growth rate?
sfdcdata 


what are our new projects won?
sfdcdata 


how are we expected to grow?
sfdcdata 


what is growth by verticals?
sfdcdata 


what is my split?
sfdcdata 


What is Market Forecast?
Gartner 




In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


classifier_response =  classifier(question, candidate_labels)
print(question)
predicted_label = classifier_response['labels'][0]
label = dataset_label.get(predicted_label)
print(label,'\n\n')



In [3]:
a_list = [1,2,3,4,5]

def get_surrounding_idx(elem,a_list):
    for index, elem in enumerate(a_list):
        if (index+1 < len(a_list) and index - 1 >= 0):
            prev_el = str(a_list[index-1])
            curr_el = str(elem)
            next_el = str(a_list[index+1])

            print(prev_el, curr_el, next_el)
            break

In [5]:
get_surrounding_idx(4,a_list)

1 2 3


In [7]:
from itertools import tee, islice, chain

def previous_and_next(some_iterable):
    prevs, items, nexts = tee(some_iterable, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return prevs, items, nexts




In [12]:
a_list = [1,2,3,4,5]

def get_surrounding_idx(item,a_list):
    # search_key = somevalue
    previous = next_ = None
    l = len(a_list)
    for index, obj in enumerate(a_list):
        if obj == item:
            if index > 0:
                previous = a_list[index - 1]
            if index < (l - 1):
                next_ = a_list[index + 1]

            return previous,next_

In [18]:
get_surrounding_idx(5,a_list)

(4, None)

In [ ]:
Index Lengths : 12 749 265 [0, 265, 321, 410, 478, 570, 571, 572, 602, 728, 739, 749] answer start idx: 749

sentance_start_idx = sentance_start_idx_list[self.find(sentance_start_idx_list,answer_start_idx)[0] - 1]
sentance_end_idx = sentance_start_idx_list [self.find(sentance_start_idx_list,answer_start_idx)[0] + 1]
                    

In [24]:
sentance_start_idx_list = [0, 265, 321, 410, 478, 570, 571, 572, 602, 728, 739, 749] # add
answer_start_idx = 739

sentance_start_idx,sentance_end_idx = get_surrounding_idx(answer_start_idx,sentance_start_idx_list)
if sentance_start_idx== None and sentance_end_idx != None:
    sentance_start_idx = sentance_start_idx_list[0]
    # print(sentance_start_idx,sentance_end_idx)
    context = text[sentance_start_idx:sentance_end_idx]
    all_context.append(context)
elif sentance_start_idx!= None and sentance_end_idx == None:
    sentance_end_idx = sentance_start_idx_list[-1]
    # print(sentance_start_idx,sentance_end_idx)
    context = text[sentance_start_idx:sentance_end_idx]
    all_context.append(context)
elif sentance_start_idx!= None and sentance_end_idx != None:
    # print(sentance_start_idx,sentance_end_idx)
    context = text[sentance_start_idx:sentance_end_idx]
    all_context.append(context)

728 749
